# Elecciones presidenciales en Twitter

Actualmente la gran parte del discurso publico se lleva a cabo en redes sociales. Por un lado esto permite a cualquier persona informarse y expresarse de manera instantánea y directa con el resto del mundo. Por el otro, permite hacer uso de tecnologías para amplificar mensajes o ideas artificialmente. 

El objetivo de este estudio es cuantificar la presencia de Bots, o cuentas automatizadas, en el discurso publico en redes sociales en el contexto de las elecciones presidenciales en Chile del año 2021.

La metodología consiste en descargar todos los mensajes que mencionen a uno de los candidatos presidenciales y analizar si los usuarios que generaron el mensaje son personas reales o cuentas automatizadas, además de a qué candidato apoyan.

## Data

In [2]:
import numpy as np
import pandas as pd

tweets_df = pd.read_json("tweets/all_tweets.json")
tweets_df

,created_at,id,author_id,text,entities
0,2021-12-01 21:27:49,1466157111570681856,3313242149,@rspg67 @abogadabruna @fbarrerarob @rob_i_c @C...,"{'mentions': [{'start': 0, 'end': 7, 'username..."
1,2021-12-01 21:27:49,1466157110824038400,1129003213204668416,RT @EduardoMadari13: MECHERO Tu le dijiste a J...,"{'mentions': [{'start': 3, 'end': 19, 'usernam..."
2,2021-12-01 21:27:49,1466157110668914688,1137473146020347904,@maylwino Señora vote por Kast para salvar al ...,"{'mentions': [{'start': 0, 'end': 9, 'username..."
3,2021-12-01 21:27:49,1466157109926584320,1309737469550698496,RT @kamiluru: @alicarina @0324Marta Desgraciad...,"{'mentions': [{'start': 3, 'end': 12, 'usernam..."
4,2021-12-01 21:27:49,1466157109536432128,142742893,RT @jenizaro11: No podia ser de otra forma! La...,"{'mentions': [{'start': 3, 'end': 14, 'usernam..."
...,...,...,...,...,...
2341605,2021-12-02 11:17:26,1466365891214381056,1047512358388748288,RT @Cooperativa: Diputado RN pro Kast declaró ...,"{'annotations': [{'start': 33, 'end': 36, 'pro..."
2341606,2021-12-02 11:17:26,1466365890270801920,708339048495063040,RT @NYC_prensa: [KAST] “El país no se construy...,"{'annotations': [{'start': 60, 'end': 73, 'pro..."
2341607,2021-12-02 11:17:26,1466365889687670784,262972427,RT @aprachile: ✅José Antonio Kast durante gira...,"{'annotations': [{'start': 16, 'end': 32, 'pro..."
2341608,2021-12-02 11:17:25,1466365885849882624,224427268,RT @jimenaorrego: Kast se juntó en USA con el ...,"{'annotations': [{'start': 18, 'end': 21, 'pro..."


In [191]:
tweets_df.author_id.unique()
print("Tweets:", len(tweets_df))
print("Usuarios:", len(tweets_df.author_id.unique()))

Tweets: 2341610
Usuarios: 172135


In [6]:
pd.to_timedelta(tweets_df.created_at.max() - tweets_df.created_at.min(), unit='D')

Timedelta('7 days 04:50:29')

In [8]:
tweets_df.created_at.apply(lambda x: x.day).unique()

array([ 1, 30, 27, 26, 29, 28,  3,  2])

## Hashtags mas populares

Partimeros nuestro analsis contandi los hashtags usados por los usarios en sus Twitts. Para limitar el analisis a menciones directas a los candidatos, solo estudiaremos hashtags que incluyan las palabras **Boric** o **Kast**:


In [289]:
def top_hashtags(df):
    hashtags_dict = {}
    for index, values in df.entities.items():
        if values and ('hashtags' in values):
            hashtags = values['hashtags']       
            for hashtag in hashtags:
                # check if hashtags is in the dictionary
                tag = "#" + hashtag['tag']
                if ("boric" in tag.lower()) or ("kast" in tag.lower()):
                    if tag in hashtags_dict:
                        hashtags_dict[tag] += 1
                    else:
                        hashtags_dict[tag] = 1
    hashtags_df = pd.DataFrame.from_dict(hashtags_dict, orient='index', columns=['count'])
    return hashtags_df.sort_values(by='count', ascending=False)

In [290]:
top_hashtags_df = top_hashtags(tweets_df)
top_hashtags_df.head(20)

,count
#Boric,45269
#BoricPresidente,35257
#Kast,31339
#MujeresPorKast,21976
#TodoChileVotaKast,19090
#KastPresidente2022,17759
#TodosKast,12656
#BoricPresidente2022,7749
#BoricNoSeAtreveDebatir,7065
#kast,6617


En esta tabla tenemos los hashtags y cuantas veces se usaron. La tabla esta ordenada por popularidad y de manera decreciente. Para nuestro analisis excluiremos los hashtags #Kast, #Boric , #kast y #boric. Esto dado que tanto un comentario positivo como negativo al respecto de los candidatos puede incluirlos y es muy dificil clasificar a quien apoyan. 

La lista de los 10 hashtags mas populares excluyendo los hastags mencionados se ve asi:

In [291]:
hashtags_list = ['#BoricPresidente',
 '#MujeresPorKast',
 '#TodoChileVotaKast',
 '#KastPresidente2022',
 '#TodosKast',
 '#BoricPresidente2022',
 '#BoricNoSeAtreveDebatir',
 '#BoricNoSeLaPuede',
 '#BoricMiente',
 '#KastPresidente']

top_hashtags_df = top_hashtags_df[top_hashtags_df.index.isin(hashtags_list)]
top_hashtags_df

,count
#BoricPresidente,35257
#MujeresPorKast,21976
#TodoChileVotaKast,19090
#KastPresidente2022,17759
#TodosKast,12656
#BoricPresidente2022,7749
#BoricNoSeAtreveDebatir,7065
#BoricNoSeLaPuede,5610
#BoricMiente,4733
#KastPresidente,4725


## Usuarios eliminados

Lo primero que llama la atencion fue que muchas cuentas ya no existen. Las cuentas dejan de existir por 3 razones:
- El usuario voluntariamente elimino su cuenta
- Twitter bloqueo la cuenta por violacion a los terminos de uso
- La cuenta ha estado inactivo por un largo periodo de tiempo 

La ultima razon la podemos descartar, dado que el analisis fue hecho posteriormente a la generacion de un Tweet.

In [299]:
users_df = pd.read_json("data/clean/users.json").drop("english", axis=1)
users_df

,id_str,screen_name,universal,status
0,1121505338690293760,None,NaN,deleted
1,1313512294815539200,KarlPop78534593,0.45,active
2,1465407053170724864,None,NaN,deleted
3,1307838136010117120,None,NaN,deleted
4,345987090,Lina1061,0.43,active
...,...,...,...,...
30722,1408048859041964032,None,NaN,deleted
30723,219678666,UnPerrit0,0.17,active
30724,68507689,Genghis__Khan,0.07,active
30725,789919502859403264,None,NaN,deleted


In [300]:
usuarios_activos = len(users_df[users_df.status == "active"])
usuarios_eliminados = len(users_df[users_df.status == "deleted"])
ratio_borrados = usuarios_eliminados / (usuarios_eliminados + usuarios_activos)

print("Total de usuarios:", len(users_df.id_str.unique()))
print("Usuarios activos:", usuarios_activos)
print("Usuarios eliminados:", usuarios_eliminados)
print(f"Proporcion de usuarios borrados: {(ratio_borrados * 100):.2f}%")

Total de usuarios: 30727
Usuarios activos: 22663
Usuarios eliminados: 8064
Proporcion de usuarios borrados: 26.24%


## Identificar si una cuenta es un Bot o una persona real

Primero debemos definir que vamos a llamar Bot. Para esto usamos un modelo llamado Botometer. Este modelo es desarollado por el ["Observatory on Social Media"](https://osome.iu.edu/), el cual es un proyecto conjunto del "Center for Complex Networks and Systems Research" y el "Network Science Institute" de la Universidad de Indiana de Estados Unidos.

Toda la informacion referente a la metodologia la pueden encontrar [aqui](https://osome.iu.edu/research/publications)

Basicamente el modelo tiene como input el `id` de una cuenta de Twitter. El programa revisa varias caracteristicas de la cuenta en evaluacion: frecuencia de poste, red de contactos, vocabulario usado, consulta a bases de datos con informacion previa sobre cuentas identificadas como bots, etc. Con esta informacion el modelo entrega varios parametros que describen la actividad de la cuenta y le asigna una probabilidad a que la cuenta sea o no un Bot

Nuestro criterio para clasificar una cuenta como bot es que el modelo le asigne una probabilidad general mayor a un 50%. Esta probabilidad es la columna `universal` en la tabla siguiente:

Por otro lado, una vista general a las cuentas clasificadas como bots:

In [301]:
usuarios_bots = len(users_df[users_df.universal > 0.5])
print("Bots:", usuarios_bots)
print(f"Proporcion de bots sobre usuarios activos: {(usuarios_bots / usuarios_activos):.2f}%")

Bots: 3303
Proporcion de bots sobre usuarios activos: 0.15%


Para desglozar esta informacion vamos a asignar los hashtags segun su apoyo: 

In [302]:
hashtags_support_dict = {
 "#BoricPresidente": "Boric",
 "#MujeresPorKast": "Kast",
 "#TodoChileVotaKast": "Kast",
 "#KastPresidente2022": "Kast",
 "#TodosKast": "Kast",
 "#BoricPresidente2022": "Boric",
 "#BoricNoSeAtreveDebatir": "Kast",
 "#BoricNoSeLaPuede": "Kast",
 "#BoricMiente": "Kast",
 "#KastPresidente": "Kast"
 }

## Usuarios por candidato

In [303]:
# asignar cuentas a candidatos
def candidate_support(text):
    for key in hashtags_support_dict:
        if key in text:
            support =  hashtags_support_dict[key]
    if "support" not in locals():
        support = np.nan
    return support 

In [304]:
tweets_df["candidate_support"] = tweets_df.text.apply(lambda x: candidate_support(x))
support_df = tweets_df[tweets_df.candidate_support.notna()]

total_top_hashtag_users = len(support_df.author_id.unique())
print("Total de usuarios de los top 10 hashtags:", total_top_hashtag_users)

Total de usuarios de los top 10 hashtags: 30572


In [305]:
users_df[~users_df.id_str.isin(support_df.author_id.unique())]

,id_str,screen_name,universal,status
7605,1055597948506177536,DoctorNo__,0.24,active
10878,728019655302897664,polletcilla,0.24,active
12119,1306975366922330112,Victori99228505,0.17,active
15738,1144616491,jrmanriquezc,0.38,active
15910,205942873,RokkuUnderwear,0.10,active
...,...,...,...,...
30407,576785263,grantokylautaro,0.11,active
30602,2899631572,wajaja_paulo,0.05,active
30667,1384506212939665408,None,NaN,deleted
30669,45174311,AleGuzmanG,0.16,active


In [310]:
tweets_df[tweets_df.author_id == 1055597948506177536]

,created_at,id,author_id,text,entities,id_str,screen_name,universal,status,support,candidate_support
330259,2021-11-27 04:20:56,1464449136648937472,1055597948506177536,RT @humber1960: Gonzalo de la Carrera. Huevada...,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
330590,2021-11-27 04:18:52,1464448614848208896,1055597948506177536,@hans_hoffmann_ Este tipo o es comunista ideol...,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
332405,2021-11-27 04:09:01,1464446134156410880,1055597948506177536,"@InterferenciaCL Ufff qué lata, que cosas feas...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
332441,2021-11-27 04:08:49,1464446083690545152,1055597948506177536,@felpohueso @diegobrowne @InterferenciaCL Ufff...,"{'annotations': [{'start': 110, 'end': 113, 'p...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
332649,2021-11-27 04:07:50,1464445839431147520,1055597948506177536,"@Mister_Wolf_0 Ufff qué lata, que cosas feas d...","{'annotations': [{'start': 83, 'end': 86, 'pro...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
332712,2021-11-27 04:07:36,1464445779066724352,1055597948506177536,"@Baltaxxar @andriwss89 Ufff qué lata, que cosa...","{'annotations': [{'start': 91, 'end': 94, 'pro...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
334636,2021-11-27 03:58:10,1464443406122459136,1055597948506177536,@Mister_Wolf_0 Shu qué terrible!! \nEl Gonzalo...,"{'annotations': [{'start': 38, 'end': 53, 'pro...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
513363,2021-11-26 19:23:44,1464313943929860096,1055597948506177536,"RT @lamonsalveg: Kast acusa aumento de ""violen...","{'annotations': [{'start': 17, 'end': 20, 'pro...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
513547,2021-11-26 19:23:14,1464313818138529792,1055597948506177536,RT @Rodrigo161277: #boric #kast esta es la tol...,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",1.055598e+18,DoctorNo__,0.24,active,False,NaN
514370,2021-11-26 19:20:56,1464313239215431680,1055597948506177536,RT @anilorac312: @biobio Boric es un mentiroso...,"{'annotations': [{'start': 25, 'end': 29, 'pro...",1.055598e+18,DoctorNo__,0.24,active,False,NaN


## Usuarios por candidato

In [275]:
kast_top_users = len(support_df[support_df.candidate_support == "Kast"].author_id.unique())
print("Usuarios que apoyan a Kast:", kast_top_users)

kast_active_users = len(support_df[(support_df.candidate_support == "Kast") & (support_df.status == "active")].author_id.unique())
print(f"Usuarios que apoyan a Kast activos: {kast_active_users} equivalente al {(kast_active_users / kast_top_users)*100:.2f}% del total de usuarios")

kast_deleted_accounts = len(support_df[(support_df.candidate_support == "Kast") & (support_df.status == "deleted")].author_id.unique())
print(f"Usuarios que apoyan a Kast eliminados: {kast_deleted_accounts} equivalente al {(kast_deleted_accounts / kast_top_users) * 100:.2f}% del total de usuarios")

kast_bot_accounts = len(support_df[(support_df.candidate_support == "Kast") & (support_df.universal > 0.5)].author_id.unique())
print(f"Bots que apoyan a Kast: {kast_bot_accounts} equivalente al {(kast_bot_accounts / kast_active_users) * 100:.2f}% de los usuarios activos")




Usuarios que apoyan a Kast: 16722
Usuarios que apoyan a Kast activos: 11705 equivalente al 70.00% del total de usuarios
Usuarios que apoyan a Kast eliminados: 5017 equivalente al 30.00% del total de usuarios
Bots que apoyan a Kast: 2762 equivalente al 23.60% de los usuarios activos


In [276]:
boric_top_users = len(support_df[support_df.candidate_support == "Boric"].author_id.unique())
print("Usuarios que apoyan a Boric:", boric_top_users)

boric_active_users = len(support_df[(support_df.candidate_support == "Boric") & (support_df.status == "active")].author_id.unique())
print(f"Usuarios que apoyan a Boric activos: {boric_active_users} equivalente al {(boric_active_users / boric_top_users)*100:.2f}% del total de usuarios")

boric_deleted_accounts = len(support_df[(support_df.candidate_support == "Boric") & (support_df.status == "deleted")].author_id.unique())
print(f"Usuarios que apoyan a Boric eliminados: {boric_deleted_accounts} equivalente al {(boric_deleted_accounts / boric_top_users) * 100:.2f}% del total de usuarios")

boric_bot_accounts = len(support_df[(support_df.candidate_support == "Boric") & (support_df.universal > 0.5)].author_id.unique())
print(f"Bots que apoyan a Boric: {boric_bot_accounts} equivalente al {(boric_bot_accounts / boric_active_users) * 100:.2f}% de los usuarios activos")

Usuarios que apoyan a Boric: 15611
Usuarios que apoyan a Boric activos: 12015 equivalente al 76.96% del total de usuarios
Usuarios que apoyan a Boric eliminados: 3596 equivalente al 23.04% del total de usuarios
Bots que apoyan a Boric: 945 equivalente al 7.87% de los usuarios activos


Cuantos tweets contienen hashtags que apoyan a cada candidato?

In [68]:
top_hashtags_df["support"] = top_hashtags_df.apply(lambda x: hashtags_support_dict[x.name], axis=1)
top_hashtags_df

/var/folders/3v/y9lrkp790dl4phjfd0r91k400000gn/T/ipykernel_27130/2123882457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_hashtags_df["support"] = top_hashtags_df.apply(lambda x: hashtags_support_dict[x.name], axis=1)


,count,support
BoricPresidente,35257,Boric
MujeresPorKast,21976,Kast
TodoChileVotaKast,19090,Kast
KastPresidente2022,17759,Kast
TodosKast,12656,Kast
BoricPresidente2022,7749,Boric
BoricNoSeAtreveDebatir,7065,Kast
BoricNoSeLaPuede,5610,Kast
BoricMiente,4733,Kast
KastPresidente,4725,Kast


In [86]:
tweets_kast = top_hashtags_df[top_hashtags_df.support == "Kast"]["count"].sum()
tweets_boric = top_hashtags_df[top_hashtags_df.support == "Boric"]["count"].sum()

kast_boric_ratio = tweets_kast / tweets_boric

print("Tweets apoyando a Kast:", tweets_kast)
print("Tweets apoyando a Boric:", tweets_boric)

print(f"Ratio de tweets Kast/Boric: {kast_boric_ratio:.2f}")

Tweets apoyando a Kast: 93614
Tweets apoyando a Boric: 43006
Ratio de tweets Kast/Boric: 2.18


Aqui podemos ver que Kast tiene 2.18 veces el numero de tweets a su favor que Boric

### Bots por hashtag

Revisemos primero cuantas cuentas borradas tiene cada hashtag.

In [145]:
# users per hashtag
hashtags_df = pd.DataFrame(columns=["hashtag", "active", "deleted", "bots"])

for hashtag in hashtags_list:
    active_users = 0
    deleted_users = 0
    bot_users = 0
    hashtag_df = tweets_df[tweets_df.text.str.contains(hashtag)]
    hashtag_users_list = list(hashtag_df.author_id.unique())
    for user in hashtag_users_list:
        users_info_df = users_df[users_df.id_str == user]
        if users_info_df.status.values[0] == "active":
            active_users += 1
        if users_info_df.status.values[0] == "deleted":
            deleted_users += 1
        if (users_info_df.universal > 0.5).values[0]:
            bot_users += 1
    hashtags_df = hashtags_df.append({
        "hashtag": hashtag,
        "active": active_users,
        "deleted": deleted_users,
        "bots": bot_users
    }, ignore_index=True)

hashtags_df

,hashtag,active,deleted,bots
0,BoricPresidente,12841,3930,1158
1,MujeresPorKast,4489,2041,1494
2,TodoChileVotaKast,4497,2061,1549
3,KastPresidente2022,4742,2115,1571
4,TodosKast,3823,1760,1393
5,BoricPresidente2022,3036,984,193
6,BoricNoSeAtreveDebatir,2342,1078,834
7,BoricNoSeLaPuede,2567,1145,988
8,BoricMiente,3515,1558,1105
9,KastPresidente,5537,2521,1749


In [148]:
hashtags_df = hashtags_df.merge(top_hashtags_df, how="left", left_on="hashtag", right_index=True)
hashtags_df

,hashtag,active,deleted,bots,count,support
0,BoricPresidente,12841,3930,1158,35257,Boric
1,MujeresPorKast,4489,2041,1494,21976,Kast
2,TodoChileVotaKast,4497,2061,1549,19090,Kast
3,KastPresidente2022,4742,2115,1571,17759,Kast
4,TodosKast,3823,1760,1393,12656,Kast
5,BoricPresidente2022,3036,984,193,7749,Boric
6,BoricNoSeAtreveDebatir,2342,1078,834,7065,Kast
7,BoricNoSeLaPuede,2567,1145,988,5610,Kast
8,BoricMiente,3515,1558,1105,4733,Kast
9,KastPresidente,5537,2521,1749,4725,Kast


### Resumen por candidato

In [202]:
total_active_users = len(users_df[users_df.status == "active"].id_str.unique())
print("Usuarios activos totales", total_active_users)
total_bots = len(users_df[users_df.universal > 0.5])
print("Bots:", total_bots)
bots_ratio = total_bots / total_active_users
print(f"Ratio de bots: {(bots_ratio * 100):.2f}%")


Usuarios activos totales 22663
Bots: 3303
Ratio de bots: 14.57%


In [149]:
kast_df = hashtags_df[hashtags_df.support == "Kast"]
kast_df

,hashtag,active,deleted,bots,count,support
1,MujeresPorKast,4489,2041,1494,21976,Kast
2,TodoChileVotaKast,4497,2061,1549,19090,Kast
3,KastPresidente2022,4742,2115,1571,17759,Kast
4,TodosKast,3823,1760,1393,12656,Kast
6,BoricNoSeAtreveDebatir,2342,1078,834,7065,Kast
7,BoricNoSeLaPuede,2567,1145,988,5610,Kast
8,BoricMiente,3515,1558,1105,4733,Kast
9,KastPresidente,5537,2521,1749,4725,Kast


In [272]:
kast_active_count = kast_df.active.sum()
kast_deleted_count = kast_df.deleted.sum()
kast_bots_count = kast_df.bots.sum()
kast_tweets_count = kast_active_count + kast_deleted_count + kast_bots_count

kast_active_ratio = kast_active_count / kast_tweets_count
kast_deleted_ratio = kast_deleted_count / kast_tweets_count
kast_bots_ratio = kast_bots_count / kast_tweets_count

print("Tweets a favor de Kast:", kast_tweets_count)
print(f"Proporcion de tweets de usuarios activos: {(kast_active_ratio * 100):.2f}%")
print(f"Proporcion de tweets de usuarios eliminados: {(kast_deleted_ratio * 100):.2f}%")
print(f"Proporcion de tweets de bots: {(kast_bots_ratio * 100):.2f}%")


Tweets a favor de Kast: 56474
Proporcion de tweets de usuarios activos: 55.80%
Proporcion de tweets de usuarios eliminados: 25.28%
Proporcion de tweets de bots: 18.92%


In [160]:
boric_df = hashtags_df[hashtags_df.support == "Boric"]
boric_df

,hashtag,active,deleted,bots,count,support
0,BoricPresidente,12841,3930,1158,35257,Boric
5,BoricPresidente2022,3036,984,193,7749,Boric


In [161]:
boric_active_count = boric_df.active.sum()
boric_deleted_count = boric_df.deleted.sum()
boric_bots_count = boric_df.bots.sum()
boric_tweets_count = boric_active_count + boric_deleted_count + boric_bots_count

bric_active_ratio = boric_active_count / boric_tweets_count
boric_deleted_ratio = boric_deleted_count / boric_tweets_count
boric_bots_ratio = boric_bots_count / boric_tweets_count

print("Tweets a favor de Boric:", boric_tweets_count)
print(f"Proporcion de tweets de usuarios activos: {(bric_active_ratio * 100):.2f}%")
print(f"Proporcion de tweets de usuarios eliminados: {(boric_deleted_ratio * 100):.2f}%")
print(f"Proporcion de tweets de bots: {(boric_bots_ratio * 100):.2f}%")

Tweets a favor de Boric: 22142
Proporcion de tweets de usuarios activos: 71.71%
Proporcion de tweets de usuarios eliminados: 22.19%
Proporcion de tweets de bots: 6.10%


## Analisis por hastag

In [178]:
def hashtag_stats(hashtag):
    hashtag_df = tweets_df[tweets_df.text.str.contains(hashtag)]
    tweets_count = len(hashtag_df)
    users_count = len(hashtag_df.author_id.unique())
    active_users_count = len(hashtag_df[hashtag_df.status == "active"])
    deleted_users_count = len(hashtag_df[hashtag_df.status == "deleted"])
    bot_users_count = len(hashtag_df[hashtag_df.universal > 0.5])

    active_users_ratio = active_users_count / tweets_count
    deleted_users_ratio = deleted_users_count / tweets_count
    bot_users_ratio = bot_users_count / active_users_count

    print("Hashtag:", hashtag)
    print("Tweets count:", tweets_count)
    print("Users count:", users_count)
    print("Active users:", active_users_count)
    print("Deeleted users:", deleted_users_count)
    print("Bot users count:", bot_users_count)
    print(f"Active users ratio: {(active_users_ratio * 100):.2f}%")
    print(f"Deleted users ratio: {(deleted_users_ratio * 100):.2f}%")
    print(f"Bot users ratio: {(bot_users_ratio * 100):.2f}%")

### Boric Hashtags

In [181]:
hashtag = "BoricPresidente"

hashtag_stats(hashtag)

Hashtag: BoricPresidente
Tweets count: 40097
Users count: 16771
Active users: 30062
Deeleted users: 10035
Bot users count: 2871
Active users ratio: 74.97%
Deleted users ratio: 25.03%
Bot users ratio: 9.55%


In [182]:
hashtag = "BoricPresidente2022"

hashtag_stats(hashtag)

Hashtag: BoricPresidente2022
Tweets count: 7643
Users count: 4020
Active users: 5711
Deeleted users: 1932
Bot users count: 400
Active users ratio: 74.72%
Deleted users ratio: 25.28%
Bot users ratio: 7.00%


### Kast Hashtags

In [183]:
hashtag = "MujeresPorKast"

hashtag_stats(hashtag)

Hashtag: MujeresPorKast
Tweets count: 21459
Users count: 6530
Active users: 14321
Deeleted users: 7138
Bot users count: 6862
Active users ratio: 66.74%
Deleted users ratio: 33.26%
Bot users ratio: 47.92%


In [184]:
hashtag = "TodoChileVotaKast"

hashtag_stats(hashtag)

Hashtag: TodoChileVotaKast
Tweets count: 18634
Users count: 6558
Active users: 12264
Deeleted users: 6370
Bot users count: 5763
Active users ratio: 65.82%
Deleted users ratio: 34.18%
Bot users ratio: 46.99%


In [185]:
hashtag = "KastPresidente2022"

hashtag_stats(hashtag)

Hashtag: KastPresidente2022
Tweets count: 17776
Users count: 6857
Active users: 11971
Deeleted users: 5805
Bot users count: 5540
Active users ratio: 67.34%
Deleted users ratio: 32.66%
Bot users ratio: 46.28%


In [186]:
hashtag = "TodosKast"

hashtag_stats(hashtag)

Hashtag: TodosKast
Tweets count: 12354
Users count: 5583
Active users: 8070
Deeleted users: 4284
Bot users count: 3767
Active users ratio: 65.32%
Deleted users ratio: 34.68%
Bot users ratio: 46.68%


In [187]:
hashtag = "BoricNoSeAtreveDebatir"

hashtag_stats(hashtag)

Hashtag: BoricNoSeAtreveDebatir
Tweets count: 7048
Users count: 3420
Active users: 4722
Deeleted users: 2326
Bot users count: 1969
Active users ratio: 67.00%
Deleted users ratio: 33.00%
Bot users ratio: 41.70%


In [188]:
hashtag = "BoricNoSeLaPuede"

hashtag_stats(hashtag)

Hashtag: BoricNoSeLaPuede
Tweets count: 5610
Users count: 3712
Active users: 3783
Deeleted users: 1827
Bot users count: 1574
Active users ratio: 67.43%
Deleted users ratio: 32.57%
Bot users ratio: 41.61%


In [189]:
hashtag = "BoricMiente"

hashtag_stats(hashtag)

Hashtag: BoricMiente
Tweets count: 9546
Users count: 5073
Active users: 6361
Deeleted users: 3185
Bot users count: 2237
Active users ratio: 66.64%
Deleted users ratio: 33.36%
Bot users ratio: 35.17%


In [190]:
hashtag = "KastPresidente"

hashtag_stats(hashtag)

Hashtag: KastPresidente
Tweets count: 22796
Users count: 8058
Active users: 15301
Deeleted users: 7495
Bot users count: 7046
Active users ratio: 67.12%
Deleted users ratio: 32.88%
Bot users ratio: 46.05%
